## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Bluff,-46.6000,168.3333,53.80,76,100,10.92,NZ,overcast clouds,2022-01-25 23:44:06
1,1,Gat,31.6100,34.7642,50.40,100,9,10.38,IL,clear sky,2022-01-25 23:44:07
2,2,Yuli,9.6989,10.2744,69.87,18,79,10.38,NG,broken clouds,2022-01-25 23:44:07
3,3,Vaini,-21.2000,-175.2000,84.36,79,20,10.36,TO,few clouds,2022-01-25 23:41:06
4,4,Ruwi,23.5996,58.5507,62.29,65,0,5.86,OM,clear sky,2022-01-25 23:44:07


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 68
What is the maximum temperature you would like for your trip? 77


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count().isnull()

City_ID                False
City                   False
Lat                    False
Lng                    False
Max Temp               False
Humidity               False
Cloudiness             False
Wind Speed             False
Country                False
Current Description    False
Date                   False
dtype: bool

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count().dropna()
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Yuli,NG,69.87,broken clouds,9.6989,10.2744,
7,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,
30,Cape Town,ZA,68.41,clear sky,-33.9258,18.4232,
36,Chuy,UY,75.67,moderate rain,-33.6971,-53.4616,
37,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,
45,Port Alfred,ZA,72.12,overcast clouds,-33.5906,26.8910,
46,East London,ZA,72.52,broken clouds,-33.0153,27.9116,
59,Cap Malheureux,MU,75.69,broken clouds,-19.9842,57.6142,
61,Urucara,BR,74.86,overcast clouds,-2.5364,-57.7600,
76,Luderitz,NaN,74.23,clear sky,-26.6481,15.1594,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key  
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Yuli,NG,69.87,broken clouds,9.6989,10.2744,
7,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,Pousada Porto Praia
30,Cape Town,ZA,68.41,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
36,Chuy,UY,75.67,moderate rain,-33.6971,-53.4616,Nuevo Hotel Plaza
37,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,Casa Tia Villa


In [37]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
686,Pagudpud,PH,76.87,scattered clouds,18.5616,120.7868,Saud Beach Resort and Hotel - 2
692,Caravelas,BR,76.71,moderate rain,-17.7125,-39.2481,Pousada dos Navegantes
707,Dzilam Gonzalez,MX,75.38,broken clouds,21.2833,-88.9333,posadas
713,Ndende,GA,71.87,light rain,-2.4008,11.3581,
718,Bahia De Caraquez,EC,75.38,light rain,-0.6000,-80.4167,Saiananda Eco-Hotel


In [38]:
clean_hotel_df["Hotel Name"].unique()

array(['', 'Pousada Porto Praia', 'Southern Sun Waterfront Cape Town',
       'Nuevo Hotel Plaza', 'Casa Tia Villa', 'The Halyards Hotel',
       'Tu Casa', 'Pereybere Hotel & Spa', 'Pousada Maria Bonita',
       'Enviro Villa', 'Redface Studios Meublés', 'Rio Xingu Camping',
       'Hotel Mirador', '39 On Nile Guest House',
       'Finch Bay Galapagos Hotel', 'Hotel Tesoro Los Cabos',
       'Halane Hotel and Restaurent', 'Aquarius Boutique Hotel Sampit',
       'Rydges Port Macquarie', 'Hotel Grand Chancellor Launceston',
       'Oumou Elghoura', 'Shanti Maurice Resort & Spa',
       "Hotel Ravin D'or", 'BUNGALOW MAHAMANINA', 'Hotel Xaguate',
       'Hotel Saigon-PT', 'Hotel Sol Nascente - Ariquemes - RO',
       'CHACARA BAILLY', 'Broadwater Mariner Resort',
       'Veranda Paul et Virginie Hotel & Spa', 'Pousada Porto De Canoas',
       'Gran Hotel Bolivar Lima', 'Shandrani Beachcomber Resort & Spa',
       'Hôtel Tshia-Muntu de Tshikapa', 'Soul Mate-Inn', 'Hôtel BUMAK',
       'At

In [40]:
clean_hotel_df.shape

(157, 7)

In [41]:
hotel_df.shape

(159, 7)

In [42]:
clean_hotel_df = clean_hotel_df[clean_hotel_df["Hotel Name"] != ""]

In [43]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,Pousada Porto Praia
30,Cape Town,ZA,68.41,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
36,Chuy,UY,75.67,moderate rain,-33.6971,-53.4616,Nuevo Hotel Plaza
37,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,Casa Tia Villa
45,Port Alfred,ZA,72.12,overcast clouds,-33.5906,26.8910,The Halyards Hotel


In [44]:
clean_hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
671,Pointe Michel,DM,76.35,clear sky,15.2500,-61.3833,Castle Comfort Lodge Dive Dominica
686,Pagudpud,PH,76.87,scattered clouds,18.5616,120.7868,Saud Beach Resort and Hotel - 2
692,Caravelas,BR,76.71,moderate rain,-17.7125,-39.2481,Pousada dos Navegantes
707,Dzilam Gonzalez,MX,75.38,broken clouds,21.2833,-88.9333,posadas
718,Bahia De Caraquez,EC,75.38,light rain,-0.6000,-80.4167,Saiananda Eco-Hotel


In [46]:
# 8a. Create the output File (CSV)
output_data_file =" WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
fig